# Sales Analysis — Course-End Project

**Overview:** This notebook follows the project brief for AAL's sales analysis. Fill `Sales.csv` in the same folder as this notebook and run the cells in order. The notebook is organized step-by-step to match the assignment.

## 1) Prepare Data for Analysis
- Import libraries
- Read `Sales.csv` into `df`
- Print shape, head, missing values

**Assumption:** `Sales.csv` is UTF-8 and contains at least columns like `Date`, `Unit`, `Sales`, `State`, `Category` (if available). If column names differ, update the code accordingly.

In [ ]:
# 1) Prepare Data for Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

# Settings for nicer plots
plt.rcParams['figure.figsize'] = (12,5)
plt.rcParams['axes.grid'] = True

# Read CSV
csv_path = "Sales.csv"   # put Sales.csv in the same directory as this notebook
df = pd.read_csv(csv_path, parse_dates=['Date'], dayfirst=False, infer_datetime_format=True)

# Basic checks
print("DataFrame shape:", df.shape)
display(df.head())
print("\nMissing values per column:")
print(df.isna().sum())

# Quick dtype check
print("\nData types:")
print(df.dtypes)

## 2) Normalize Data for Analysis
Normalize `Unit` and `Sales` using MinMax scaling to range [0,1]. Create `df_dataonly` and `normalize_data` as requested.

In [ ]:
# 2) Normalize Unit and Sales columns
# Ensure columns exist
for col in ['Unit','Sales']:
    if col not in df.columns:
        raise KeyError(f"Required column '{col}' not found in dataframe. Available columns: {list(df.columns)}")

df_dataonly = df[['Unit','Sales']].copy()

scaler = MinMaxScaler()
normalize_data = scaler.fit_transform(df_dataonly)  # ndarray with 2 columns

# Convert back to DataFrame for convenience
df_normalized = pd.DataFrame(normalize_data, columns=['Unit_norm','Sales_norm'])
print("normalize_data shape:", normalize_data.shape)
print("Min and max of each normalized column:")
print(df_normalized.agg(['min','max']))

# attach normalized columns to original df (optional)
df[['Unit_norm','Sales_norm']] = df_normalized[['Unit_norm','Sales_norm']]
display(df.head())

## 3) Visualize Overall Trends
Plot daily aggregated Unit and Sales over time (Date vs Unit, Date vs Sales).

In [ ]:
# 3) Visualize overall trends (daily sum)
# Ensure Date is datetime
df['Date'] = pd.to_datetime(df['Date'])

daily = df.groupby(df['Date'].dt.date).agg({'Unit':'sum','Sales':'sum'}).reset_index()
daily['Date'] = pd.to_datetime(daily['Date'])

# Plot Unit
plt.figure()
plt.plot(daily['Date'], daily['Unit'], marker='.', linewidth=1)
plt.title('Daily Units Sold (sum per day)')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.tight_layout()

# Plot Sales
plt.figure()
plt.plot(daily['Date'], daily['Sales'], marker='.', linewidth=1)
plt.title('Daily Sales Revenue (sum per day)')
plt.xlabel('Date')
plt.ylabel('Sales Revenue')
plt.tight_layout()

# Show last few rows of aggregated daily
display(daily.tail())

## 4) Analyze Monthly Data
Chunk quarterly data into monthly slices (example: October, November, December). Adjust date ranges based on dataset. The example below assumes dates in 2025-10-01 to 2025-12-31 or similar.

In [ ]:
# 4) Chunk into monthly data
# Example: determine min and max dates then create monthly ranges automatically
min_date = df['Date'].min()
max_date = df['Date'].max()
print("Date range in dataset:", min_date.date(), "to", max_date.date())

# Create month keys and monthly DataFrames in a dict
df['YearMonth'] = df['Date'].dt.to_period('M')
months = sorted(df['YearMonth'].unique())
print("Detected Year-Month periods:", months)

monthly_dfs = {str(m): df[df['YearMonth']==m].copy() for m in months}

# Example: show describe for each month
for k,v in monthly_dfs.items():
    print("\n---", k, "---")
    display(v.describe(include='all'))

## 5) Describe Data
Use `describe()` on overall and per-month datasets (done in previous cell).

## 6) Analyze Unit and Sales Distributions
Boxplots for Unit and Sales per month.

In [ ]:
# 6) Boxplots: Units and Sales per month
import matplotlib.dates as mdates

months_list = list(monthly_dfs.keys())
units_by_month = [monthly_dfs[m]['Unit'].dropna() for m in months_list]
sales_by_month = [monthly_dfs[m]['Sales'].dropna() for m in months_list]

# Boxplot Units
plt.figure(figsize=(10,5))
plt.boxplot(units_by_month, labels=months_list, showmeans=True)
plt.title('Units sold — distribution per month')
plt.xlabel('Month')
plt.ylabel('Units')
plt.tight_layout()

# Boxplot Sales
plt.figure(figsize=(10,5))
plt.boxplot(sales_by_month, labels=months_list, showmeans=True)
plt.title('Sales Revenue — distribution per month')
plt.xlabel('Month')
plt.ylabel('Sales Revenue')
plt.tight_layout()

## 7) Monthly Plots and Analysis
Plot Units and Sales for each month and a consolidated 3-month plot.

In [ ]:
# 7) Monthly plots
for m in months_list:
    d = monthly_dfs[m].groupby(monthly_dfs[m]['Date'].dt.date).agg({'Unit':'sum','Sales':'sum'}).reset_index()
    d['Date'] = pd.to_datetime(d['Date'])
    plt.figure()
    plt.plot(d['Date'], d['Unit'], marker='o', label='Units')
    plt.plot(d['Date'], d['Sales'], marker='x', label='Sales')
    plt.title(f'Units and Sales over days — {m}')
    plt.xlabel('Date')
    plt.legend()
    plt.tight_layout()

# Consolidated 3-month sums (if exactly 3 months present)
if len(months_list) <= 6:  # won't create overly busy plot
    plt.figure()
    for m in months_list:
        d = monthly_dfs[m].groupby(monthly_dfs[m]['Date'].dt.date).agg({'Sales':'sum'}).reset_index()
        d['Date'] = pd.to_datetime(d['Date'])
        plt.plot(d['Date'], d['Sales'], marker='.', label=str(m))
    plt.title('Consolidated monthly Sales (daily lines)')
    plt.xlabel('Date')
    plt.legend()
    plt.tight_layout()

## 8) Comprehensive Snapshot
Create summary tables: totals per month, average order value, units per order (if OrderID exists).

In [ ]:
# 8) Snapshot summaries
# Totals per month
summary_month = df.groupby(df['YearMonth']).agg(Total_Units=('Unit','sum'), Total_Sales=('Sales','sum'), Avg_Unit=('Unit','mean'), Avg_Sales=('Sales','mean')).reset_index()
display(summary_month)

# If OrderID exists, compute Units per Order and Sales per Order
if 'OrderID' in df.columns:
    order_summary = df.groupby('OrderID').agg(Units_per_order=('Unit','sum'), Sales_per_order=('Sales','sum')).describe()
    print("\nOrder-level summary:")
    display(order_summary)
else:
    print("\nNo 'OrderID' column found — skipping order-level metrics.")

## 9) Statewise Sales
Aggregate sales and units by State and visualize top states.

In [ ]:
# 9) Statewise analysis
if 'State' in df.columns:
    state_summary = df.groupby('State').agg(Total_Units=('Unit','sum'), Total_Sales=('Sales','sum')).reset_index()
    state_summary = state_summary.sort_values('Total_Sales', ascending=False)
    display(state_summary.head(15))
    
    # Bar plot top 10 states by sales
    top10 = state_summary.head(10)
    plt.figure(figsize=(10,5))
    plt.bar(top10['State'], top10['Total_Sales'])
    plt.title('Top 10 States by Sales')
    plt.xlabel('State')
    plt.ylabel('Total Sales')
    plt.xticks(rotation=45)
    plt.tight_layout()
else:
    print("No 'State' column found. If you have location data in another column, update the column name in the code.")

## 10) Groupwise Analysis
Examples: Category wise, Sub-Category, Store, or Channel. Update column names if your data uses other labels.

In [ ]:
# 10) Groupwise analysis examples
for col in ['Category','SubCategory','Store','Channel']:
    if col in df.columns:
        grp = df.groupby(col).agg(Total_Units=('Unit','sum'), Total_Sales=('Sales','sum')).reset_index().sort_values('Total_Sales', ascending=False)
        print(f"\n--- Top 8 for {col} ---")
        display(grp.head(8))

## 11) Timewise Analysis
Examples: weekday vs weekend, day-of-week patterns, hourly patterns (if timestamp present).

In [ ]:
# 11) Timewise analysis
df['Weekday'] = df['Date'].dt.day_name()
weekday_summary = df.groupby('Weekday').agg(Total_Units=('Unit','sum'), Total_Sales=('Sales','sum')).reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
display(weekday_summary)

plt.figure()
plt.plot(weekday_summary.index, weekday_summary['Total_Sales'], marker='o')
plt.title('Sales by Day of Week')
plt.xlabel('Day')
plt.ylabel('Total Sales')
plt.tight_layout()

# If Time or Hour column exists, do hourly analysis
if 'Time' in df.columns:
    # try parse time column
    try:
        df['Time_parsed'] = pd.to_datetime(df['Time']).dt.time
        df['Hour'] = pd.to_datetime(df['Time']).dt.hour
    except Exception as e:
        print('Could not parse Time column:', e)
if 'Hour' in df.columns:
    hour_summary = df.groupby('Hour').agg(Total_Sales=('Sales','sum'), Total_Units=('Unit','sum')).reset_index()
    display(hour_summary)
    plt.figure()
    plt.plot(hour_summary['Hour'], hour_summary['Total_Sales'], marker='o')
    plt.title('Sales by Hour of Day')
    plt.xlabel('Hour')
    plt.ylabel('Total Sales')
    plt.tight_layout()

## Conclusions & Recommendations
Write data-driven findings here after running the notebook: inventory suggestions, top-performing states/categories, monthly seasonality, and anomalies. Include action items like marketing focus for low-performing categories or inventory rebalancing for high-demand states.